In [1]:
from dateutil.rrule import rrule, DAILY
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import requests

In [4]:
df_original = pd.read_csv('impressions_reach_CJ_instagram_21-2-7_to_23-1-24_corected_version.csv')
df_likes_and_comments = pd.read_csv('CJ_Instagram_metrics_(impressions,reach,likes,comments)_2021-1_to_2023-1.csv')

In [13]:
df_original.tail()

,Unnamed: 0,impressions,date,reach
615,819,722,2023-01-25,517
616,820,227,2023-01-26,208
617,821,289,2023-01-27,188
618,822,2781,2023-01-28,2258
619,823,861,2023-01-29,764


In [12]:
df_likes_and_comments.tail()

,Unnamed: 0,impressions,date,reach,comments,likes
716,716,998,2023-01-17,627,0,0
717,717,859,2023-01-18,597,2,4
718,718,746,2023-01-19,504,0,1
719,719,729,2023-01-20,463,0,1
720,720,768,2023-01-21,490,0,6


In [14]:
result = pd.concat([df_original, df_likes_and_comments], axis=1).reindex(df_original.date)

In [17]:
result.shape

(620, 10)

In [19]:
df1 = df_original.set_index(df_original.date)
df2 = df_likes_and_comments.set_index(df_likes_and_comments.date)

In [21]:
result = pd.concat([df1, df2], axis=0)


In [25]:
result.drop('Unnamed: 0', axis=1)

,impressions,date,reach,comments,likes
date,,,,,
2021-02-07,1438,2021-02-07,1294,NaN,NaN
2021-02-08,1195,2021-02-08,1078,NaN,NaN
2021-02-09,451,2021-02-09,348,NaN,NaN
2021-02-10,108,2021-02-10,99,NaN,NaN
2021-02-11,163,2021-02-11,31,NaN,NaN
...,...,...,...,...,...
2023-01-17,998,2023-01-17,627,0.0,0.0
2023-01-18,859,2023-01-18,597,2.0,4.0
2023-01-19,746,2023-01-19,504,0.0,1.0


In [26]:
result= result.fillna(0)

In [28]:
result.tail()

,Unnamed: 0,impressions,date,reach,comments,likes
date,,,,,,
2023-01-17,716,998,2023-01-17,627,0.0,0.0
2023-01-18,717,859,2023-01-18,597,2.0,4.0
2023-01-19,718,746,2023-01-19,504,0.0,1.0
2023-01-20,719,729,2023-01-20,463,0.0,1.0
2023-01-21,720,768,2023-01-21,490,0.0,6.0


In [31]:
result.to_csv('CJ Instagram DATA 21-02-17 to 23-01-21.csv')

In [16]:
tokken = "EAAmNvp7ZCK7kBANpm3sdMScQF7oAqdxcZBYU2u1GkZCqgygcgncYwhxjjAlYmyyY2DcvtKgp6scYuEZBYS4EKh1y0basQH5YRUMzge4PCT3PWSZC0xtWgbi5FUJukuMhbHAq7obwaI9rcUAlZArgZB2PJzaFPAL1abuEVjeY0kHqwvsW5BE0ONjmZANw8TtwtBKHE3URvwASMQZDZD"
start = '23-03-26'
end = '23-03-27'
metric = 'impressions,reach'

In [8]:
def fetch_data(start,end, metric,tokken):
    url= f"https://graph.facebook.com/v15.0/17841401901416928/insights?metric={metric}&period=day&since={start}&until={end}&access_token={tokken}"
    r = requests.get(url)
    json=r.json()
    df_impressions = pd.DataFrame(json['data'][0]['values'])
    df_impressions.columns = ['impressions','time']
    df_reach = pd.DataFrame(json['data'][1]['values'])
    df_reach.columns = ['reach','time']
    df_final =pd.concat([df_impressions, df_reach],axis=1)
    df_final= df_final[['impressions','reach']]
    
    return df_final

In [17]:
def test_token(start,end, metric,tokken):
    url= f"https://graph.facebook.com/v15.0/17841401901416928/insights?metric={metric}&period=day&since={start}&until={end}&access_token={tokken}"
    r = requests.get(url)
    json=r.json()

    
    return r


In [18]:
test_token(start,end, metric,tokken)

<Response [200]>

In [4]:
def fetch_data_impressions_reach(start,end,tokken):
    url= f"https://graph.facebook.com/v15.0/17841401901416928/insights?metric=likes,comments&period=day&metric_type=total_value&since={start}&until={end}&access_token={tokken}"
    r = requests.get(url)
    json=r.json()
    likes = json['data'][0]['total_value']['value']
    comments = json['data'][1]['total_value']['value']
    return likes, comments

In [101]:
fetch_data_impressions_reach(start, end, tokken)

(13, 0)

In [5]:
def date_range_df(entry_date):
    # get the actual date
    actual_date = pd.to_datetime('today').normalize()

    # create a date range between the entry date and the actual date
    date_range = pd.date_range(start=entry_date, end=actual_date, freq='D')

    # create a DataFrame with the date range
    df = pd.DataFrame({'start_date': date_range, 'end_date': date_range + pd.Timedelta(days=1)})

    return df

In [48]:
date_range_df('21-01-2023')

,start_date,end_date
0,2023-01-21,2023-01-22
1,2023-01-22,2023-01-23
2,2023-01-23,2023-01-24
3,2023-01-24,2023-01-25
4,2023-01-25,2023-01-26
...,...,...
62,2023-03-24,2023-03-25
63,2023-03-25,2023-03-26
64,2023-03-26,2023-03-27
65,2023-03-27,2023-03-28


In [6]:
def fetch_all_data(entry_date, token, metric):
    date_range = date_range_df(entry_date)
    df_final = pd.DataFrame(columns=['start_date', 'reach', 'impressions', 'likes', 'comments'])

    for index, row in date_range.iterrows():
        start = row['start_date'].strftime('%Y-%m-%d')
        end = row['end_date'].strftime('%Y-%m-%d')
        impressions_reach = fetch_data_impressions_reach(start, end, token)
        data = fetch_data(start, end, metric, token)
        df_final = df_final.append({'start_date': row['start_date'], 'reach': data['reach'][0], 'impressions': data['impressions'][0], 'likes': impressions_reach[0], 'comments': impressions_reach[1]}, ignore_index=True)

    return df_final

In [105]:
df_final_to_csv = fetch_all_data('21-01-2023', tokken, metric)

In [107]:
df_final_to_csv.to_csv('CJ Instagram DATA 23-01-21 to 23-03-28 .csv')

In [11]:
df_final_to_csv = fetch_all_data('28-03-2023', tokken, metric)

In [12]:
df_final_to_csv

,start_date,reach,impressions,likes,comments
0,2023-03-28,221,499,6,0
1,2023-03-29,151,195,4,0
2,2023-03-30,77,174,0,0
3,2023-03-31,706,831,24,2
4,2023-04-01,324,439,3,0
5,2023-04-02,198,228,4,0
6,2023-04-03,276,300,6,0
7,2023-04-04,49,50,0,0


In [23]:
df_for_april = fetch_all_data('31-03-2023', tokken, metric)

In [24]:
df_for_april.head(20)


,start_date,reach,impressions,likes,comments
0,2023-03-31,706,831,24,2
1,2023-04-01,324,439,3,0
2,2023-04-02,198,228,4,0
3,2023-04-03,276,300,6,0
4,2023-04-04,157,165,4,0
5,2023-04-05,109,134,3,0
6,2023-04-06,375,514,16,0
7,2023-04-07,2904,3811,268,1
8,2023-04-08,1126,1351,106,1
9,2023-04-09,569,647,39,1


In [25]:
df_for_april.to_csv('CJ Instagram DATA 31-03-2023 to 02-05-2023.csv')